# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.3.1'

### Importar o dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [4]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [59]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train, X_test, y_train_sig, y_test_sig = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train, X_test, y_train_tanh, y_test_tanh = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train, X_test, y_train_elu, y_test_elu = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [60]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train= sc.fit_transform(X_train)
X_test= sc.transform(X_test)
X_train_sig= sc.fit_transform(X_train)
X_test_sig= sc.transform(X_test)
X_train_tanh= sc.fit_transform(X_train)
X_test_tanh= sc.transform(X_test)
X_train_elu= sc.fit_transform(X_train)
X_test_elu = sc.transform(X_test)


In [61]:
np.shape(X_train)
np.shape(X_train_sig)
np.shape(X_train_tanh)
np.shape(X_train_elu)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [62]:
ann = tf.keras.models.Sequential()#RELU
ann1 = tf.keras.models.Sequential()#SIGMÓIDE
ann2 = tf.keras.models.Sequential()#TANH
ann3 = tf.keras.models.Sequential()#ELU = 'EXPONENCIAL LINEAR UNITY'

### Adding the input layer and the first hidden layer

In [63]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))#RELU
ann1.add(tf.keras.layers.Dense(units=6, activation='sigmoid'))#SIGMOIDE
ann2.add(tf.keras.layers.Dense(units=6, activation='tanh'))#TANH
ann3.add(tf.keras.layers.Dense(units=6, activation='elu'))#ELU

### Adding the second hidden layer

In [64]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann1.add(tf.keras.layers.Dense(units=6, activation='sigmoid'))#SIGMOIDE
ann2.add(tf.keras.layers.Dense(units=6, activation='tanh'))#TANH
ann3.add(tf.keras.layers.Dense(units=6, activation='elu'))#ELU

### Adding the output layer

In [65]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))#1-RELU, 2-RELU,3-SIGMOID
ann1.add(tf.keras.layers.Dense(units=1, activation='tanh'))#1-SIGMOID, 2-SIGMOID,3-TANH
ann2.add(tf.keras.layers.Dense(units=1, activation='elu'))#1-TANH, 2-TANH,3-ELU
ann3.add(tf.keras.layers.Dense(units=1, activation='relu'))#1-ELU, 2-ELU,3-RELU

## Part 3 - Training the ANN

### Compiling the ANN

In [66]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann1.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann3.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [67]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)


Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5541 - accuracy: 0.7629
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4804 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4598 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4434 - accuracy: 0.7962
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4267 - accuracy: 0.7979
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4111 - accuracy: 0.8081
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3964 - accuracy: 0.8149
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3837 - accuracy: 0.8175
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3752 - accuracy: 0.8192
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3695 - accura

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [68]:
y_pred = ann.predict(X_test)
y_pred

array([[0.23827922],
       [0.40750125],
       [0.2015768 ],
       ...,
       [0.34453934],
       [0.15577394],
       [0.1835595 ]], dtype=float32)

In [69]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [70]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1502   93]
 [ 195  210]]


In [ ]:
SIGMOIDE

In [71]:
ann1.fit(X_train_sig, y_train_sig, batch_size = 32, epochs = 100)



Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5980 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4869 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4553 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4443 - accuracy: 0.7964
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4416 - accuracy: 0.7979
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4413 - accuracy: 0.8001
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4394 - accuracy: 0.8041
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4550 - accuracy: 0.8060
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4362 - accuracy: 0.8026
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4390 - accura

In [72]:
y_pred1 = ann1.predict(X_test_sig)
y_pred1

array([[0.29144987],
       [0.2835193 ],
       [0.23600927],
       ...,
       [0.18785611],
       [0.18071231],
       [0.31172687]], dtype=float32)

In [73]:
y_pred1 = (y_pred1 > 0.5)
y_pred1

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [74]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test_sig, y_pred1)
print(cm)

[[1560   35]
 [ 283  122]]


In [ ]:
TANH

In [75]:
ann2.fit(X_train_tanh, y_train_tanh, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 1.3473 - accuracy: 0.7639
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 1.0334 - accuracy: 0.7624
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.8685 - accuracy: 0.7656
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7089 - accuracy: 0.7491
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.6098 - accuracy: 0.7631
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.6203 - accuracy: 0.7398
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5836 - accuracy: 0.7570
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5412 - accuracy: 0.7666
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5209 - accuracy: 0.7731
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5052 - accura

In [76]:
y_pred2 = ann2.predict(X_test_tanh)
y_pred2

array([[0.36174196],
       [0.28622934],
       [0.16294275],
       ...,
       [0.14369631],
       [0.17331687],
       [0.1860709 ]], dtype=float32)

In [77]:
y_pred2 = (y_pred2 > 0.5)
y_pred2

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [78]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test_tanh, y_pred2)
print(cm)

[[1563   32]
 [ 244  161]]


In [ ]:
ELU

In [79]:
ann3.fit(X_train_elu, y_train_elu, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 2.4898 - accuracy: 0.6532
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 1.5550 - accuracy: 0.7054
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 1.1408 - accuracy: 0.7254
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.7669 - accuracy: 0.7412
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5939 - accuracy: 0.7624
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5434 - accuracy: 0.7688
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5208 - accuracy: 0.7736
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4958 - accuracy: 0.7772
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4877 - accuracy: 0.7820
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4813 - accura

In [80]:
y_pred3 = ann3.predict(X_test_elu)
y_pred3

array([[0.26318017],
       [0.38292658],
       [0.14367607],
       ...,
       [0.21477565],
       [0.17962343],
       [0.30819544]], dtype=float32)

In [81]:
y_pred3 = (y_pred3 > 0.5)
y_pred3

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [83]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test_elu, y_pred3)
print(cm)

[[1541   54]
 [ 243  162]]


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)